In [ ]:
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# import datetime as dt
# import yfinance as yf
# import pandas_ta as ta
# import mplfinance as mpf
# from IPython.display import display
# from tvDatafeed import TvDatafeed, Interval

# # get credentials for tradingview
# username = 'Joaqtrades'
# password = '4671926Trading?'

# # initialize tradingview
# tv = TvDatafeed(username, password)

directory = "/Users/JoaqMac/Desktop/Python_Trading/Strategy_1/iteration_performance/"

# Define tickers and parameters
tickers = ["AAPL", "ZM", "INTC", "PEP", "NFLX", "GOOG", "ADBE"] #"NFLX" #"ZM" #"COST" #"ADBE" #"BKNG" #"META" #"AMD" #"SBUX" #"DPZ"  #"LOW" #"CSCO" #"LYFT" #"BABA" #"PTON" #"MRNA" #"BIDU"
numbs_trigger_values = [2, 3, 4]
numbs_enter_values = [1, 2, 3]
numbs_exit_values = [0.5, 1, 3]
numbs_rr_values = [4, 8, 12]

# Other constants
exchange = 'NASDAQ'
years = 5
interval = Interval.in_4_hour
sma_length = 25
avg_window = 6
commissions = 4
initial_funds = 10000
percentage = 0.02

# Iterate over each ticker
for ticker in tickers:
    performance_list = []
    
    bars = round((years * 252) * 2) # 2*252
    stock_data = tv.get_hist(symbol=ticker, exchange=exchange, interval=interval, n_bars=bars)

    # Drop the columns I won't be using:
    stock_data = stock_data.drop(columns=['open', 'volume'])
    
    # Reset the index without dropping it, and rename the index column
    stock_data.reset_index(drop=False, inplace=True)
    stock_data.rename(columns={'index': 'index'}, inplace=True)

    # Add a unique reference column with the order of the rows
    stock_data['id'] = range(1, len(stock_data) + 1)

    # Reorder the columns to move reference_id to the left
    columns = ['id'] + [col for col in stock_data.columns if col != 'id']
    stock_data = stock_data[columns]

    # Step 2: Calculate the SMA
    sma = ta.sma(stock_data['close'], length=sma_length)

    # Step 3: Add the SMA as a new column to the dataframe
    stock_data['sma'] = sma

    # Creating the above - below sma column
    stock_data['ab_bel_sma'] = np.where(stock_data['low'] >= stock_data['sma'], 'above', 'BELOW')

    # Initialize the 'ref_line' column with NaN values
    stock_data['ref_line'] = np.nan

    # Iterate through the DataFrame starting from the second row
    for i in range(1, len(stock_data)):
        current_ab_bel_sma = stock_data.at[i, 'ab_bel_sma']
        previous_ab_bel_sma = stock_data.at[i - 1, 'ab_bel_sma']
        if current_ab_bel_sma == 'BELOW' and previous_ab_bel_sma == 'above':
            stock_data.at[i, 'ref_line'] = stock_data.at[i, 'sma']
        else:
            stock_data.at[i, 'ref_line'] = stock_data.at[i - 1, 'ref_line']

    # Calculate the percentage difference
    stock_data['distance'] = round((stock_data['low'] - stock_data['ref_line']) / stock_data['ref_line'] * 100, 2)

    def min_negative(values):
        negatives = values[values < 0]
        return negatives.min() if len(negatives) > 0 else 0

    def max_positive(values):
        positives = values[values > 0]
        return positives.max() if len(positives) > 0 else 0

    # Group by 'ref_line' and apply the min_negative and max_positive functions to 'distance'
    result_neg = stock_data.groupby('ref_line')['distance'].agg(min_negative).reset_index().rename(columns={'distance': 'neg_distance'})
    result_pos = stock_data.groupby('ref_line')['distance'].agg(max_positive).reset_index().rename(columns={'distance': 'pos_distance'})

    # Merge the results into a single DataFrame
    result = pd.merge(result_neg, result_pos, on='ref_line')

    # Filter stock_data where 'sma' matches 'ref_line'
    filtered_stock_data = stock_data[stock_data['sma'] == stock_data['ref_line']]

    # Merge to add the 'id' column where 'sma' matches 'ref_line'
    result = pd.merge(result, filtered_stock_data[['ref_line', 'id']], on='ref_line', how='left')

    # Sort values by 'id'
    result = result.sort_values(by='id')

    # Calculate the rolling averages excluding the current row
    result['avg_neg_distance'] = result['neg_distance'].shift(1).rolling(window=avg_window).mean().fillna(0)
    result['avg_pos_distance'] = result['pos_distance'].shift(1).rolling(window=avg_window).mean().fillna(0)

    # Replace the cells with 0 by NaN
    result.replace(0.00, np.nan, inplace=True)

    # Merge the dataframes based on the 'ref_line' column
    stock_data = stock_data.merge(result[['ref_line', 'avg_neg_distance', 'avg_pos_distance']], on='ref_line', how='left')

    for numbs_trigger in numbs_trigger_values:
        for numbs_enter in numbs_enter_values:
            for numbs_exit in numbs_exit_values:
                for numbs_rr in numbs_rr_values:
                    
                    # Set the value of the different numbs
                    enter_perc = round(stock_data['avg_neg_distance'] * numbs_enter, 2)
                    trigger_perc = round(enter_perc * numbs_trigger, 2)
                    exit_perc = round(stock_data['avg_pos_distance'] * numbs_exit, 2)
                    stop_perc = round(((stock_data['avg_neg_distance'] * numbs_enter) - (stock_data['avg_pos_distance'] * numbs_exit)) / numbs_rr + (stock_data['avg_neg_distance'] * numbs_enter), 2)

                    # Replace the cells with 0 by NaN
                    stock_data.replace(0.00, np.nan, inplace=True)

                    # Create the new columns
                    stock_data['enter'] = round(stock_data['ref_line'] - (-stock_data['ref_line'] * enter_perc / 100), 2)
                    stock_data['trigger'] = round(stock_data['ref_line'] - (-stock_data['ref_line'] * trigger_perc / 100), 2)
                    stock_data['exit'] = round(stock_data['ref_line'] - (-stock_data['ref_line'] * exit_perc / 100), 2)
                    stock_data['stop'] = round(stock_data['ref_line'] - (-stock_data['ref_line'] * stop_perc / 100), 2)

                    # Initialize the trigger_status column based on the initial condition
                    stock_data['trigger_status'] = stock_data.apply(lambda row: 'ON' if row['low'] <= row['trigger'] else 'OFF', axis=1)

                    # Iterate through the rows to ensure trigger_status stays 'on' until ref_line changes
                    previous_ref_line = None
                    trigger_status = 'off'
                    for index, row in stock_data.iterrows():
                        if row['ref_line'] != previous_ref_line:
                            trigger_status = 'ON' if row['low'] <= row['trigger'] else 'OFF'
                        else:
                            if trigger_status == 'ON':
                                trigger_status = 'ON'
                            else:
                                trigger_status = 'ON' if row['low'] <= row['trigger'] else 'OFF'
                        stock_data.at[index, 'trigger_status'] = trigger_status
                        previous_ref_line = row['ref_line']

                    stock_data['previous_trigger_status'] = stock_data['trigger_status'].shift(1)

                    # Initialize the enter_1 column based on the initial condition
                    stock_data['enter_1'] = stock_data.apply(
                        lambda row: 'ON' if row['trigger_status'] == "ON" and row['previous_trigger_status'] == "ON" and row['high'] >= row['enter'] else 'OFF', axis=1)

                    # Iterate through the rows to ensure enter_1 stays 'on' until ref_line changes
                    previous_ref_line = None
                    enter_1 = 'OFF'
                    for index, row in stock_data.iterrows():
                        if row['ref_line'] != previous_ref_line:
                            enter_1 = 'ON' if row['trigger_status'] == "ON" and row['previous_trigger_status'] == "ON" and row['high'] >= row['enter'] else 'OFF'
                        else:
                            if enter_1 == 'OFF':
                                enter_1 = 'ON' if row['trigger_status'] == "ON" and row['previous_trigger_status'] == "ON" and row['high'] >= row['enter'] else 'OFF'
                        stock_data.at[index, 'enter_1'] = enter_1
                        previous_ref_line = row['ref_line']

                    # Initialize the enter_2 column based on the initial condition
                    stock_data['enter_2'] = stock_data.apply(
                        lambda row: 'ON' if row['trigger_status'] == "ON" and row['previous_trigger_status'] == "ON" and row['low'] <= row['enter'] else 'OFF', axis=1)

                    # Iterate through the rows to ensure enter_2 stays 'on' until ref_line changes
                    previous_ref_line = None
                    enter_2 = 'OFF'
                    for index, row in stock_data.iterrows():
                        if row['ref_line'] != previous_ref_line:
                            enter_2 = 'ON' if row['trigger_status'] == "ON" and row['previous_trigger_status'] == "ON" and row['low'] <= row['enter'] else 'OFF'
                        else:
                            if enter_2 == 'OFF':
                                enter_2 = 'ON' if row['trigger_status'] == "ON" and row['previous_trigger_status'] == "ON" and row['low'] <= row['enter'] else 'OFF'
                        stock_data.at[index, 'enter_2'] = enter_2
                        previous_ref_line = row['ref_line']

                    stock_data = stock_data.drop(columns=['previous_trigger_status'])

                    # Initialize the position_status column based on the initial condition
                    stock_data['position_status'] = stock_data.apply(
                        lambda row: 'ON' if row['trigger_status'] == "ON" and row['enter_1'] == "ON" and row['enter_2'] == "ON" else 'OFF', axis=1)

                    # Iterate through the rows to ensure position_status stays 'on' until ref_line changes
                    previous_ref_line = None
                    position_status = 'OFF'
                    for index, row in stock_data.iterrows():
                        if row['ref_line'] != previous_ref_line:
                            position_status = 'ON' if row['trigger_status'] == "ON" and row['enter_1'] == "ON" and row['enter_2'] == "ON" else 'OFF'
                        else:
                            if position_status == 'OFF':
                                position_status = 'ON' if row['trigger_status'] == "ON" and row['enter_1'] == "ON" and row['enter_2'] == "ON" else 'OFF'
                        stock_data.at[index, 'position_status'] = position_status
                        previous_ref_line = row['ref_line']

                    # Initialize the outcome column to NaN
                    stock_data['rr'] = np.nan

                    # Process each ref_line series
                    unique_ref_lines = stock_data['ref_line'].unique()
                    for ref_line in unique_ref_lines:
                        ref_line_data = stock_data[stock_data['ref_line'] == ref_line]
                        outcome_set = False
                        for i in ref_line_data.index:
                            if not outcome_set and stock_data.at[i, 'position_status'] == 'ON':
                                if stock_data.at[i, 'high'] >= stock_data.at[i, 'exit'] and stock_data.at[i, 'low'] <= stock_data.at[i, 'stop']:
                                    stock_data.at[i, 'rr'] = -1
                                    outcome_set = True
                                elif stock_data.at[i, 'high'] >= stock_data.at[i, 'exit']:
                                    stock_data.at[i, 'rr'] = numbs_rr
                                    outcome_set = True
                                elif stock_data.at[i, 'low'] <= stock_data.at[i, 'stop']:
                                    stock_data.at[i, 'rr'] = -1
                                    outcome_set = True
                                elif i == ref_line_data.index[-1]:
                                    stock_data.at[i, 'rr'] = (stock_data.at[i, 'enter'] - stock_data.at[i, 'low']) / (stock_data.at[i, 'stop'] - stock_data.at[i, 'enter'])
                                    outcome_set = True
                                else:
                                    stock_data.at[i, 'rr'] = np.nan

                    stock_data['rr'] = stock_data['rr'].round(2)

                    # Initialize the columns
                    stock_data['gross_pnl'] = initial_funds
                    stock_data['net_pnl'] = np.nan

                    # Update pnl_alone and gross_pnl based on rr values
                    for i in range(1, len(stock_data)):
                        if not np.isnan(stock_data.at[i, 'rr']):
                            stock_data.at[i, 'gross_pnl'] = round(((stock_data.at[i - 1, 'gross_pnl'] * percentage) * stock_data.at[i, 'rr'])) + stock_data.at[i - 1, 'gross_pnl']
                            stock_data.at[i, 'net_pnl'] = round((stock_data.at[i - 1, 'gross_pnl'] * percentage) * stock_data.at[i, 'rr'] - (((stock_data.at[i - 1, 'gross_pnl'] * percentage) / stock_data.at[i, 'low']) * commissions), 2)
                        else:
                            stock_data.at[i, 'gross_pnl'] = stock_data.at[i - 1, 'gross_pnl']
                            stock_data.at[i, 'net_pnl'] = np.nan

                    # Compute derived metrics
                    gross_pnl = stock_data['gross_pnl'].iloc[-1]
                    avg_rr = round(stock_data['rr'].mean(), 2)

                    # Compute Win Rate
                    wins = (stock_data['net_pnl'] > 0).sum()
                    trades_amount = stock_data['net_pnl'].count()
                    win_rate = wins / trades_amount

                    # Compute Profit Factor
                    positive_pnls = stock_data[stock_data['net_pnl'] > 0]['net_pnl'].sum()
                    if positive_pnls == 0:
                        positive_pnls = 1
                    negative_pnls = abs(stock_data[stock_data['net_pnl'] < 0]['net_pnl'].sum())
                    if negative_pnls == 0:
                        negative_pnls = 1
                    profit_factor = round(positive_pnls / negative_pnls if negative_pnls != 0 else 0, 2)

                    # Compute P/L Ratio
                    average_positive_pnl = stock_data[stock_data['net_pnl'] > 0]['net_pnl'].mean()
                    average_negative_pnl = abs(stock_data[stock_data['net_pnl'] < 0]['net_pnl'].mean())

                    # Compute Max Drawdown
                    max_drawdown = stock_data['gross_pnl'].min()

                    # Compute additional metrics
                    wins_amount = (stock_data['net_pnl'] > 0).sum()
                    loss_amount = (stock_data['net_pnl'] < 0).sum()
                    avg_won = average_positive_pnl
                    avg_lost = -average_negative_pnl
                    total_net_pnl = round(stock_data['net_pnl'].sum())
                    twrrr = round(trades_amount * win_rate * avg_rr, 2)
                    profitable_stocks = 1 if total_net_pnl > 0 else 0
                    trading_days = (stock_data['avg_neg_distance'].notna().sum()) / 2
                    daysintrade = (stock_data['position_status'] == 'ON').sum()

                    # Create the performance dictionary
                    performance_data = {
                        'ticker': ticker,
                        'years': round(trading_days / 252, 2),
                        'comm': commissions,
                        'gross_pnl': gross_pnl,
                        'net_pnl': total_net_pnl,
                        'avg_rr': avg_rr,
                        'twrrr': twrrr,
                        'profit_factor': profit_factor,
                        'max_drawdown': max_drawdown,
                        'avg_won': avg_won,
                        'avg_lost': avg_lost,
                        'wins_amount': wins_amount,
                        'loss_amount': loss_amount,
                        'trades_amount': trades_amount,
                        'avg_daysintrade': round(daysintrade / trades_amount, 2),
                        'avg_window': avg_window,
                        'n_trigger': numbs_trigger,
                        'n_enter': numbs_enter,
                        'n_exit': numbs_exit,
                        'n_rr': numbs_rr,
                        'profitable_stocks': profitable_stocks
                    }

                    performance_list.append(performance_data)

    # Convert the performance list to a DataFrame
    performance_df = pd.DataFrame(performance_list)
    
    # Add an index column
    performance_df.index = performance_df.index + 1
    performance_df.reset_index(inplace=True)
    performance_df.rename(columns={'index': 'index'}, inplace=True)

    # Save the performance DataFrame to a CSV file
    filename = f"{directory}{ticker}_performance.csv"
    performance_df.to_csv(filename, index=False)

    
from IPython.display import Audio, display
    
# Function to play a sound
def play_sound():
    display(Audio(url='https://cdn.freesound.org/previews/539/539574_3775755-lq.mp3', autoplay=True))

# Play the sound
play_sound()
